<a href="https://colab.research.google.com/github/yuaquintero/DeepLearning/blob/master/%5BTALLER%5D_Customized_loss_functions_and_regularization_with_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading the Fashion MNIST database...

In [0]:
import os
import gzip
import numpy as np
import matplotlib.pyplot as plt
import warnings; warnings.simplefilter('ignore')

In [0]:
from tensorflow.keras import datasets
(x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()
X_train = x_train.reshape(x_train.shape[0],x_train.shape[1]*x_train.shape[2])
X_test = x_test.reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2])

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
pip install tensorflow==1.13rc0

In [0]:
!pip install keras

In [0]:
import tensorflow as tf
from keras.utils import np_utils
from sklearn.preprocessing import StandardScaler

input_dim = X_train.shape[1]

scaler = StandardScaler()
X_trainN = scaler.fit_transform(X_train)
X_testN = scaler.transform(X_test)

# convert list of labels to binary class matrix
y_trainOHE = np_utils.to_categorical(y_train)
nb_classes = y_trainOHE.shape[1]

Using TensorFlow backend.


## Exercises

### 1.1 Regularization

1.1.1 Define a new model using the keras sequential API. The model must have four hidden layers with the following neurons [128,64,32,16]. For all the hidden layers use the 'relu' activation function.

In [0]:
from numpy.random import seed
from tensorflow.keras import regularizers
seed(1)
#tf.random.set_seed(2)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(128, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_classes, activation='softmax'))


Run the following cell to train and test the model.

In [0]:
from tensorflow.keras import optimizers

# or instantiate an optimizer before passing it to model.compile
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['acc'])
history= model.fit(X_trainN[:500,:], y_trainOHE[:500,:], epochs=1000, batch_size=16, validation_split=0, verbose=0)

preds = model.predict(X_testN, verbose=0)
preds = np.argmax(preds,axis=1)
Accuracy = np.mean(preds == y_test)
print('Accuracy = ', Accuracy*100, '%')

Accuracy =  78.46 %


Create a graph with the histogram of the network weigths in the first hidden layer

In [0]:
weight_0=model.layers[0].get_weights()[0]
len(weight_0)
weight_0.shape[0]
#np.squeeze(W)


784

In [0]:
plt.imshow(weight_0-model.layers[0].get_weights()[0], cmap='gray');

In [0]:
plot_value_array(weight_0-model.layers[0].get_weights()[0], cmap='gray')

In [0]:
len(model.layers)

In [0]:
weights=model.layers[2].get_weights()[0]

print(weights)

[[ 0.11882945 -0.01498369 -0.09627794 ... -0.18894364  0.15332371
  -0.10029481]
 [-0.14417073 -0.00185425 -0.09237389 ... -0.18656796  0.06119299
  -0.12092488]
 [ 0.08765567 -0.05115098  0.01132738 ... -0.01604713  0.0366763
  -0.04896652]
 ...
 [ 0.10220526 -0.04267174 -0.05175226 ...  0.02114982  0.00586397
   0.03611135]
 [-0.07716034  0.1200362   0.26986492 ... -0.07792313 -0.10976321
   0.15365523]
 [-0.06025179 -0.1370014   0.21084313 ...  0.10295838 -0.04754204
  -0.00142164]]


In [0]:
plt.....

1.1.2 Modify the former model to include $L_2$ regularization to every layer of the former model. Define a regularization parameter equal to 0.0001. 

In [0]:
seed(1)
tf.random.set_seed(2)

#del model
#....

Run the following cell to train and test the model

In [0]:
# or instantiate an optimizer before passing it to model.compile
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)
model.fit(X_trainN[:500,:], y_trainOHE[:500,:], epochs=1000, batch_size=16, validation_split=0, verbose=0)

preds = model.predict(X_testN, verbose=0)
preds = np.argmax(preds,axis=1)
Accuracy = np.mean(preds == y_test)
print('Accuracy = ', Accuracy*100, '%')

Create a graph with the histogram of the network weigths in the first hidden layer. Compare it with the histogram obtained in the previous exercise. Is there any effect due to the regularization?

In [0]:
plt.....

1.1.3 Define a new model using the keras sequential API including $L_1$ and $L_2$ regularization methods for every layer. Define a regularization parameter equal to 0.0001 for both regularization terms.

In [0]:
seed(1)
tf.random.set_seed(2)

del model
....

Run the following cell to train the model and estimate the accuracy.

In [0]:
# or instantiate an optimizer before passing it to model.compile
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)
model.fit(X_trainN[:500,:], y_trainOHE[:500,:], epochs=1000, batch_size=16, validation_split=0, verbose=0)

preds = model.predict(X_testN, verbose=0)
preds = np.argmax(preds,axis=1)
Accuracy = np.mean(preds == y_test)
print('Accuracy = ', Accuracy*100, '%')

Create a graph with the histogram of the network weigths in the first hidden layer. Compare it with the histograms obtained in the previous exercises. What is the effect of applying $L_1$ regularization?

### 1.2 Customized loss function

In [0]:
#Example MSE
from tensorflow.keras import backend as K

def mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=-1)

1.2.1 Use the backend component of keras (https://keras.io/backend/) to define the following loss function and use it to train the model.

$$\mathcal{L}({\bf{\hat{y}}},{\bf{y}}) = -\frac{1}{N}\sum_{i=1}^N \sum_{j=1}^C {\bf{1}}_{y_i \in C_j} w_{j}\log p_{model}[y_i \in C_j]$$

which corresponds to a weighted version of the categorical cross entropy loss function.

In [0]:
from tensorflow.keras import backend as K
def weighted_categorical_crossentropy(weights):
    ...

        
    def loss(y_true, y_pred):
        ...
        return loss
    
    return loss

1.2.2 Use the weighted categorical cross entropy function to train the MLP model with 3 layers defined at the begining of this document. Use the following weights = np.array([1,1,1,1,1,1,4,1,1,1]). Evaluate the model with the test dataset and plot the confusion matrix. 

In [0]:
weights = np.array([1,1,1,1,1,1,4,1,1,1])

In [0]:
del model
...

Train and validate the model. Compare the confusion matrix obtained using the weighted loss function with one obtained in the former class.

In [0]:
model.fit...

cm = confusion...

...
plt.imshow(cm, interpolation='nearest', cmap=cmap)
plt.title('Normalized confusion matrix')
